# Short Answer Grading DNN

In [1]:
"""
# Imports and inits

"""


%matplotlib inline
import matplotlib.pyplot as plt

import collections
import re
import random
import numpy as np
import pickle
from nltk import sent_tokenize

# from nltk.tokenize import sent_tokenize, word_tokenize
import keras
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop, Adam
from keras import backend as K

from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import tensorflow as tf

N_SEQ = 58
N_VOCAB = 20000
EMBED_DIM = 300

unk_vec = np.zeros(EMBED_DIM)


"""

Load STS (semantic text similarity) model trained on SemEval dataset

"""


# define pearson correlation as model metric

def pearson_corr_neg(a, b):

    a_mean = K.mean(a)
    b_mean = K.mean(b)
    a_diff = a - a_mean
    b_diff = b - b_mean

    pcorr = K.sum(a_diff*b_diff)/K.sqrt(K.sum(a_diff*a_diff))/K.sqrt(K.sum(b_diff*b_diff))
    
    return -pcorr

fname_model = './models/RNN-sts-max-pc.hdf5'
model = load_model(fname_model, custom_objects={'pearson_corr_neg': pearson_corr_neg})

Using TensorFlow backend.


In [16]:
def compute_and_print_score(x1_text, x2_text):

    x1_sent = sent_tokenize(x1_text)
    n_x1_sent = len(x1_sent)

    x2_sent = sent_tokenize(x2_text)
    n_x2_sent = len(x2_sent)

    # copy all previous preprocessing steps here:

    # Merge list of sentences
    # x1_sent is list of sentence string
    # x1_seq is list of list of integer indices

    x_sent_all = x1_sent + x2_sent
    if len(x_sent_all) == 1:
        x_sent_all = x_sent_all + ['the.'] # add dummy sentence so that keras fit_on_texts word_tokenize instead of char_tokenize

    # Step 1 : build dict_tmp

    # Note : can't set weights for keras embedding layer
    # extract dictionary of unique words
    Ktokenizer = Tokenizer(num_words=N_VOCAB)
    Ktokenizer.fit_on_texts(x_sent_all) 
    dict_tmp = Ktokenizer.word_index
    #print(dict_tmp)
    #print("----------------------")

    # Step 2 : Load model dictionary into dict_K

    def save_obj(obj, name):
        with open(name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    def load_obj(name):
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)

    dict_K = load_obj('../Proj_Me1/data/dict_K')
    #print("Length of dict_K :", len(dict_K))
    #print(len(dict_K), list(dict_K.keys())[0:20])


    # Step 3 : Extend dict_K with new vocab from dict_tmp

    # Note : dictionary assignment assigns pointer not copy content!!!

    tmpkeys = list(dict_tmp.keys())
    Kkeys = list(dict_K.keys())
    i = len(dict_K) + 1
    #print("New word(s) not in previous dataset model trained on:")
    for w in tmpkeys:
        if w not in Kkeys:
            #print(w)
            dict_K[w] = i
            i = i + 1
    #print("Length of dict_K :", len(dict_K))
    save_obj(dict_K, './data/dict_K+')
    Kvals = list(dict_K.values())
    Kitems = list(dict_K.items())
    #print(Kitems[15347:15350])


    # Step 4 : Format text into input for model
    sseq_tmp = Ktokenizer.texts_to_sequences(x_sent_all)

    # Step 5 : Need to translate word indices derived from dict_tmp 
    # to indices derived from dict_K
    sseq=sseq_tmp
    for i,seq_tmp in enumerate(sseq_tmp):
        seq=seq_tmp
        for j, wj in enumerate(seq_tmp):
            # translate dict_tmp index to index in dict_K; 
            # note that index is one more than the dictionary index, but same as dictionary value
            seq[j] = dict_K[tmpkeys[wj-1]]
        sseq[i] = seq

    # Step 6 : Pad sequences

    sseq_pad = pad_sequences(sseq, maxlen=N_SEQ)
    #print(len(sseq_pad[0]))

    # Step 7 : Make exhaustive pairwise combos

    x1 = sseq_pad[:n_x1_sent]
    x2 = sseq_pad[n_x1_sent:]
    x1_inp = np.zeros((n_x1_sent*n_x2_sent,N_SEQ),dtype=x1.dtype)
    x2_inp = np.zeros((n_x1_sent*n_x2_sent,N_SEQ),dtype=x1.dtype)
    #print("x1:",type(x1), x1.shape, x1.dtype)
    #print("x2:",type(x2), x2.shape, x2.dtype)
    #print("x1_inp:",type(x1_inp), x1_inp.shape, x1_inp.dtype)
    for i in range(n_x1_sent):
        for j in range(n_x2_sent):
            n = i*n_x2_sent+j
            #print("n,i,j:",n,i,j)
            x1_inp[n] = x1[i]
            x2_inp[n] = x2[j]

    """
    print("x1:\n",x1)
    print("x1_inp:\n",x1_inp)

    print("-------")
    print("x2:\n",x2)
    print("x2_inp:\n",x2_inp)
    """


    # Step 8 : Format input if necessary
    # test if it is a single sentence input
    # if x1_inp or x2_inp contain single sentence, 
    # need to reshape x1_inp or x2_inp to expected list of lists input

    if isinstance(x1[0], np.int32):
        x1_inp = np.reshape(x1_inp,(1,N_SEQ))
    if isinstance(x2[0], np.int32):
        x2_inp = np.reshape(x2_inp,(1,N_SEQ))



    # Step 9 : Compute score

    y_pred = model.predict([x1_inp, x2_inp], verbose=0, steps=None)
    #print(y_pred)

    """
    Aggregate similarity score is computed assuming first text is 
    expected correct answer and second is student answer to be graded.

    Aggregate similarity score is obtained by :
    1. For each sentence in first text (Text1_Sent_i), 
       take the maximum similarity scores with every sentence
       in second text (Text2_Sent_1, Text2_Sent_2 etc) to obtain
       "hits" in student's answer with respect to expected answer content
       in sentence Text1_Sent_i. 

    2. Sum up normalized scores in step 2 and 
       divide by number of sentences in first text (model answer text)

    """
    
    m = 0.0
    for i in range(n_x1_sent):
        m += np.max(y_pred[i*n_x2_sent:(i+1)*n_x2_sent])

#    print("Expected answer :\n%s\n" % x1_text)
#    print("Student answer  : %s" % x2_text)
    print("Aggregate score = %.2f" % (m/n_x1_sent))



In [23]:
"""
Read in input text
and sentence_tokenize text into 2 lists of strings: x1_sent and x2_sent
followed by merging both lists into x_sent_all
"""
sample =[]
sample.append('When the cover was dipped into hot water, the metal cover gained heat '\
              'from the hot water and expanded. It expanded more than the glass container. '\
              'Because metal is a better conductor of heat than glass.  This reduced '\
              'the friction between cover of the jam jar. Thus allowing him to twist the '\
              'cover off with less force.')
sample.append('When the cover was dipped into the basin of hot water, the metal cover gained '\
              'heat from the hot water. It expanded more than the glass container because '\
              'metal is a better conductor of heat than glass. This reduced the friction '\
              'between cover of the jam jar.')
sample.append('The metal cover expanded more than the glass container and so he '\
              'can open the jam jar.')
sample.append('The metal cover became lose because of less friction.')
sample.append('The hot water made the metal cover expand and so he was able to twist '\
              'the cover open.')

print("Sample answers :")
for i,s in enumerate(sample):
    print(i,":", s,'\n')

ans_true = input("Please input an expected answer or choose 1 from above :")
if len(ans_true)==1:
    ans_true = sample[int(ans_true)]

ans = input("Please input a student answer or choose 1 from above :")
if len(ans)==1:
    ans = sample[int(ans)]


compute_and_print_score(ans_true, ans)

Sample answers :
0 : When the cover was dipped into hot water, the metal cover gained heat from the hot water and expanded. It expanded more than the glass container. Because metal is a better conductor of heat than glass.  This reduced the friction between cover of the jam jar. Thus allowing him to twist the cover off with less force. 

1 : When the cover was dipped into the basin of hot water, the metal cover gained heat from the hot water. It expanded more than the glass container because metal is a better conductor of heat than glass. This reduced the friction between cover of the jam jar. 

2 : The metal cover expanded more than the glass container and so he can open the jam jar. 

3 : The metal cover became lose because of less friction. 

4 : The hot water made the metal cover expand and so he was able to twist the cover open. 

Please input an expected answer or choose 1 from above :0
Please input a student answer or choose 1 from above :1
Aggregate score = 3.47
